In [ ]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import numpy as np
import matplotlib.pyplot as plt
from scipy import io
import seaborn as sns
sns.set_style("darkgrid")
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from time import time
from tqdm import tqdm
from matplotlib import cm
import h5py
DTYPE = 'float32'
filename = '../data/ITER_like_equilibrium_dataset.h5'

In [ ]:
# # download datasets from gdrive, # uncomment if you want to download the dataset
# import gdown
# gdown.download(id="1-5KP7_OYIvDD_QXvIr5sDihVxZx1qJCN", output='data/ITER_like_equilibrium_dataset_sample.mat', quiet=False)
# gdown.download(id="1Gn_OrMzxPRkTk-i77--HiWmWZyd8i8ue", output='data/ITER_like_equilibrium_dataset.mat', quiet=False)

In [15]:
# # generate h5 file from downloaded mat file # uncomment if needed
# data = io.loadmat('../data/ITER_like_equilibrium_dataset.mat')
# with h5py.File(filename,'w') as handle:
#     for k,v in data.items():
#         if '__' not in k:
#             print(k)
#             chunks = [1]
#             chunks.extend(list(v.shape[1:]))
#             handle.create_dataset(k, data=v, shape=v.shape, chunks=tuple(chunks), dtype='float64')

In [ ]:
# utils functions
import numpy as np
from scipy.interpolate import interp2d

def sample_random_subgrids(RR_pixels,ZZ_pixels,nr=64,nz=64):
  delta_r_min = .33*(RR_pixels.max()-RR_pixels.min())
  delta_r_max = .75*(RR_pixels.max()-RR_pixels.min())
  delta_z_min = .2*(ZZ_pixels.max()-ZZ_pixels.min())
  delta_z_max = .75*(ZZ_pixels.max()-ZZ_pixels.min())
  delta_r = np.random.uniform(delta_r_min,delta_r_max,1)
  r0 = np.random.uniform(RR_pixels.min(),RR_pixels.min()+delta_r_max-delta_r,1)
  delta_z = np.random.uniform(delta_z_min,delta_z_max,1)
  z0 = np.random.uniform(ZZ_pixels.min(),ZZ_pixels.min()+delta_z_max-delta_z,1)
  rr = np.linspace(r0,r0+delta_r,nr)
  zz = np.linspace(z0,z0+delta_z,nz)
  rr_grid, zz_grid = np.meshgrid(rr,zz,indexing='xy')
  return rr_grid, zz_grid

def get_box_from_grid(rr_grid, zz_grid):
  return np.array([
      [rr_grid.min(), zz_grid.min()],
      [rr_grid.max(), zz_grid.min()],
      [rr_grid.max(), zz_grid.max()],
      [rr_grid.min(), zz_grid.max()],
      [rr_grid.min(), zz_grid.min()]])

def get_grid_from_box(box,nr=64,nz=64):
  rr = np.linspace(box[:,0].min(),box[:,0].max(),nr)
  zz = np.linspace(box[:,1].min(),box[:,1].max(),nz)
  rr_grid, zz_grid = np.meshgrid(rr,zz,indexing='xy')
  return rr_grid, zz_grid

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 

def interp_fun(f,RR_pixels,ZZ_pixels,rr_grid,zz_grid,kind='quintic'):
  x_pts = RR_pixels[0,:].ravel()
  y_pts = ZZ_pixels[:,0].ravel()
  # interp_func = RegularGridInterpolator((x_pts, y_pts), f.T,kind)
  interp_func = interp2d(x_pts, y_pts, f,kind=kind)
  f_int = interp_func(rr_grid[0,:].ravel(),zz_grid[:,0].ravel()).reshape(rr_grid.shape)
  return f_int

def def_grids_and_interp(f,rhs,RR_pixels,ZZ_pixels,nr=64,nz=64,kind='quintic'):
  rr_grid, zz_grid = sample_random_subgrids(RR_pixels,ZZ_pixels,nr,nz)
  f_grid = interp_fun(f,RR_pixels,ZZ_pixels,rr_grid, zz_grid,kind=kind)
  rhs_grid = interp_fun(rhs,RR_pixels,ZZ_pixels,rr_grid, zz_grid,kind=kind)
  return rr_grid, zz_grid, f_grid, rhs_grid

In [16]:
n_sample_train = 35000

with h5py.File(filename, 'r') as handle:
    DB_psi_pixel_test_ConvNet = handle['DB_psi_pixel_test_ConvNet'][()].astype(DTYPE)
    DB_meas_Bpickup_test_ConvNet = handle['DB_meas_Bpickup_test_ConvNet'][()].astype(DTYPE)
    DB_coils_curr_test_ConvNet = handle['DB_coils_curr_test_ConvNet'][()].astype(DTYPE)
    DB_p_test_ConvNet = handle['DB_p_test_ConvNet'][()].astype(DTYPE)
    RR_pixels = handle['RR_pixels'][()].astype(DTYPE)
    ZZ_pixels = handle['ZZ_pixels'][()].astype(DTYPE)
    DB_res_RHS_pixel_test_ConvNet = handle['DB_res_RHS_pixel_test_ConvNet'][()].astype(DTYPE)
    DB_separatrix_200_test_ConvNet = handle['DB_separatrix_200_test_ConvNet'][()].astype(DTYPE)
    DB_Jpla_pixel_test_ConvNet = handle['DB_Jpla_pixel_test_ConvNet'][()].astype(DTYPE)
y_data = DB_psi_pixel_test_ConvNet
X_data = np.column_stack((DB_meas_Bpickup_test_ConvNet, DB_coils_curr_test_ConvNet)) #, DB_f_test_ConvNet, DB_p_test_ConvNet ))
# Save RR_pixels, ZZ_pixels
outdir = '../data/'
dict_save = {'RR_pixels':RR_pixels,'ZZ_pixels':ZZ_pixels}
nr,nz = RR_pixels.shape
io.savemat('{}data_geo_Dataset_NeuralOpt_super_res_{}x{}.mat'.format(outdir,nr,nz),dict_save)   

In [ ]:
# plot dataset example
for i in range(0,1):
    ind_plot = np.random.randint(0,DB_Jpla_pixel_test_ConvNet.shape[0],1)[0]
    fig, axs = plt.subplots(1, 2, figsize=(9, 3), sharey=True)
    img = axs[0].contour(RR_pixels,ZZ_pixels,DB_psi_pixel_test_ConvNet[ind_plot,:,:],15)
    fig.colorbar(img)
    axs[0].plot(DB_separatrix_200_test_ConvNet[ind_plot,:,0],DB_separatrix_200_test_ConvNet[ind_plot,:,1],c='g')
    axs[0].axis('equal')
    axs[0].set_xlabel('r [m]')
    axs[0].set_ylabel('z [m]')
    axs[0].set_title('Ψ [Wb] - equil. #{}'.format(ind_plot))
    img = axs[1].contourf(RR_pixels,ZZ_pixels,DB_Jpla_pixel_test_ConvNet[ind_plot,:,:],15)
    fig.colorbar(img)
    axs[1].axis('equal')
    axs[1].set_title('$J_Ψ$ [A/m2] - equil. #{}'.format(ind_plot))
    axs[1].set_xlabel('r [m]')
    axs[1].set_ylabel('z [m]')
    axs[1].plot(DB_separatrix_200_test_ConvNet[ind_plot,:,0],DB_separatrix_200_test_ConvNet[ind_plot,:,1],c='g')

In [ ]:
# plot Ψ and GS operator
cmap = cm.inferno
for i in range(0,1):
    ind_plot = np.random.randint(0,DB_Jpla_pixel_test_ConvNet.shape[0],1)[0]
    fig, axs = plt.subplots(1, 2, figsize=(9, 3), sharey=True)
    img = axs[0].contourf(RR_pixels,ZZ_pixels,DB_psi_pixel_test_ConvNet[ind_plot,:,:],15,cmap=cmap)
    axs[0].plot( DB_separatrix_200_test_ConvNet[ind_plot,:,0], DB_separatrix_200_test_ConvNet[ind_plot,:,1], c='g')
    axs[0].set_xlim([RR_pixels.min(),RR_pixels.max()])
    axs[0].set_ylim([ZZ_pixels.min(),ZZ_pixels.max()])
    axs[0].axis('equal')
    axs[0].set_axis_off()
    axs[0].set_title('Ψ')
    qq = DB_res_RHS_pixel_test_ConvNet[ind_plot,:,:]
    img = axs[1].contourf(RR_pixels,ZZ_pixels,qq,15,cmap=cmap)
    axs[1].set_title('GS operator')
    axs[1].axis('equal')
    axs[1].set_xlim([RR_pixels.min(),RR_pixels.max()])
    axs[1].set_ylim([ZZ_pixels.min(),ZZ_pixels.max()])
    axs[1].axis('equal')
    axs[1].plot(DB_separatrix_200_test_ConvNet[ind_plot,:,0], DB_separatrix_200_test_ConvNet[ind_plot,:,1], c='g')
    axs[1].set_axis_off()
    # plt.savefig('./figures/equil_and_GSope', dpi=300)

In [19]:
### train-test split
id_train = np.load('{}/id_train.npy'.format(outdir))
id_test = np.load('{}/id_test.npy'.format(outdir))
### Standardize input data
scaler = StandardScaler()
X_data = scaler.fit_transform(X_data)

In [ ]:
f = y_data[0,:,:]
rr_grid, zz_grid = sample_random_subgrids(RR_pixels,ZZ_pixels)
box = get_box_from_grid(rr_grid, zz_grid)
f_grid = interp_fun(f, RR_pixels, ZZ_pixels, rr_grid, zz_grid, kind='quintic')

fig,ax = plt.subplots(1,3)
ax[0].scatter(RR_pixels.ravel(), ZZ_pixels.ravel(), marker='.')
ax[0].scatter(rr_grid.ravel(), zz_grid.ravel(), marker='.')
ax[0].set_aspect('equal', 'box')

im = ax[1].contourf(RR_pixels, ZZ_pixels, f, 20)
ax[1].set_aspect('equal', 'box')
ax[1].plot(box[:,0],box[:,1])
plt.colorbar(im,ax=ax[1])

im = ax[2].contourf(rr_grid, zz_grid, f_grid, 20)
ax[2].set_aspect('equal', 'box')
ax[2].plot(box[:,0],box[:,1])
plt.colorbar(im,ax=ax[2])
plt.show()

In [ ]:
'''
The full training dataset is ~60k equilibria. We select 35k equilibria on the full grid
and 35k equilibria to be resampled on randomly generated sub-grids. These two datasets 
are then put together ans shuffled.

_all_domain -> quantities related to the original grid
_super_res -> quantities related to random sub-grids

It woul be nice to explore how the performance changes using data different from the 
aforementioned 35k/35k.
'''
np.random.seed(42)
ind_all_domain = np.random.choice(id_train,n_sample_train)
ind_super_res = np.random.choice(id_train,n_sample_train)

n_all_domain = ind_all_domain.shape[0]
n_super_res = ind_super_res.shape[0]

print(f'{n_all_domain} equil on the original grid')
print(f'{n_super_res} equil on random sub-grids')

y_train_all_domain = y_data[ind_all_domain,:,:]
X_train_all_domain = X_data[ind_all_domain,:]
res_RHS_pixel_train_all_domain = DB_res_RHS_pixel_test_ConvNet[ind_all_domain,:,:]

y_test_all_domain = y_data[id_test,:,:]
X_test_all_domain = X_data[id_test,:]
res_RHS_pixel_test_all_domain = DB_res_RHS_pixel_test_ConvNet[id_test,:,:]

f_for_super_res = y_data[ind_super_res,:,:]
rhs_for_super_res = DB_res_RHS_pixel_test_ConvNet[ind_super_res,:,:]

X_train_super_res = X_data[ind_super_res,:]
y_train_super_res = np.zeros((n_super_res,64,64), dtype=DTYPE)
rhs_train_super_res = np.zeros_like(y_train_super_res, dtype=DTYPE)
RR_grid_super_res = np.zeros_like(y_train_super_res, dtype=DTYPE)
ZZ_grid_super_res = np.zeros_like(y_train_super_res, dtype=DTYPE)

In [22]:
# n_parallel = os.cpu_count()

# t_start = time()
# rr_grid, zz_grid, f_grid, rhs_grid = zip(
#     *Parallel(n_jobs=n_parallel)(delayed(def_grids_and_interp)(
#         f_i,rhs_i,RR_pixels,ZZ_pixels,64,64
#         ) for f_i,rhs_i in tqdm(zip(f_for_super_res,rhs_for_super_res)))
# )

# t_elapsed = time() - t_start
# print(f't_elapsed = {t_elapsed}, time_per_step = {t_elapsed/n_super_res}')

In [ ]:
t_start = time()
for i in tqdm(range(f_for_super_res.shape[0]), miniters = 0):
    rr_grid, zz_grid, f_grid, rhs_grid = def_grids_and_interp(f_for_super_res[i,...],rhs_for_super_res[i,...],RR_pixels,ZZ_pixels)
    RR_grid_super_res[i,:,:] = rr_grid
    ZZ_grid_super_res[i,:,:] = zz_grid
    y_train_super_res[i,:,:] = f_grid
    rhs_train_super_res[i,:,:] = rhs_grid
t_elapsed = time() - t_start
print(f't_elapsed = {t_elapsed}, time_per_step = {t_elapsed/n_super_res}')
RR_grid_all_domain = np.tile(RR_pixels,(n_all_domain,1,1))
ZZ_grid_all_domain = np.tile(ZZ_pixels,(n_all_domain,1,1))

In [ ]:
### Check re-sampled equils
i = 0

f = f_for_super_res[i,:,:]
rhs = rhs_for_super_res[i,:,:]
rr_grid = RR_grid_super_res[i,:,:]
rr_grid = RR_grid_super_res[i,:,:]
rr_grid = RR_grid_super_res[i,:,:]
rr_grid = RR_grid_super_res[i,:,:]
box = get_box_from_grid(rr_grid, zz_grid)

fig,ax = plt.subplots(1,3)
ax[0].scatter(RR_pixels.ravel(), ZZ_pixels.ravel(), marker='.')
# ax[0].scatter(rr_grid.ravel(), zz_grid.ravel(), marker='.')
ax[0].plot(box[:,0],box[:,1],c='r')
ax[0].set_aspect('equal', 'box')
im = ax[1].contourf(RR_pixels, ZZ_pixels, f, 20)
ax[1].set_aspect('equal', 'box')
ax[1].plot(box[:,0],box[:,1],c='r')
plt.colorbar(im,ax=ax[1])
im = ax[2].contourf(rr_grid, zz_grid, f_grid, 20)
ax[2].set_aspect('equal', 'box')
ax[2].plot(box[:,0],box[:,1])
plt.colorbar(im,ax=ax[2])
plt.show()

fig,ax = plt.subplots(1,3)
ax[0].scatter(RR_pixels.ravel(), ZZ_pixels.ravel(), marker='.')
# ax[0].scatter(rr_grid.ravel(), zz_grid.ravel(), marker='.')
ax[0].plot(box[:,0],box[:,1],c='r')
ax[0].set_aspect('equal', 'box')
im = ax[1].contourf(RR_pixels, ZZ_pixels, rhs, 20)
ax[1].set_aspect('equal', 'box')
ax[1].plot(box[:,0],box[:,1],c='r')
plt.colorbar(im,ax=ax[1])
im = ax[2].contourf(rr_grid, zz_grid, rhs_grid, 20)
ax[2].set_aspect('equal', 'box')
ax[2].plot(box[:,0],box[:,1])
plt.colorbar(im,ax=ax[2])
plt.show()

In [ ]:
# compute kernels for the GS equation train
from ..src.train.utils_train import calc_laplace_df_dr_ker
y_train = np.row_stack([y_train_all_domain,y_train_super_res])
X_train = np.row_stack([X_train_all_domain,X_train_super_res])
res_RHS_pixel_train = np.row_stack([res_RHS_pixel_train_all_domain,rhs_train_super_res])
RR_pixel_train = np.row_stack([RR_grid_all_domain,RR_grid_super_res])
ZZ_pixel_train = np.row_stack([ZZ_grid_all_domain,ZZ_grid_super_res])
print(X_train.shape, y_train.shape, res_RHS_pixel_train[:,1:-1,1:-1].shape, RR_pixel_train.shape, ZZ_pixel_train.shape)
''' Compute kernels for GS Equation for the qeuils on the sub-grids (dr,dz is grid-dependent), 
otherwise we would have to compute them on the fly during each training step '''
Laplace_kernel = np.zeros((y_train.shape[0],3,3),dtype=DTYPE)
Df_dr_kernel = np.zeros((y_train.shape[0],3,3),dtype=DTYPE)
hrs, hzs = RR_pixel_train[:,1,2] - RR_pixel_train[:,1,1], ZZ_pixel_train[:,2,1] - ZZ_pixel_train[:,1,1]
for i in tqdm(range(y_train.shape[0])):
    hr = RR_pixel_train[i,1,2] - RR_pixel_train[i,1,1]
    hz = ZZ_pixel_train[i,2,1] - ZZ_pixel_train[i,1,1] 
    alfa = -2*(hr**2 + hz**2)
    Laplace_kernel_i = np.array(([0, hr**2/alfa, 0], [hz**2/alfa, 1, hz**2/alfa], [0, hr**2/alfa, 0]))
    Df_dr_kernel_i = np.array(([0, 0, 0], [+1, 0, -1], [0, 0, 0]))/(2*hr*alfa)*(hr**2*hz**2)
    Laplace_kernel[i,:,:] = Laplace_kernel_i
    Df_dr_kernel[i,:,:] = Df_dr_kernel_i
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # filter out everything but errors
### Dataset with equilibria on the entire domain
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train, res_RHS_pixel_train[:,1:-1,1:-1], RR_pixel_train, ZZ_pixel_train, Laplace_kernel, Df_dr_kernel)).shuffle(42)#.batch(batch_size=1024)
tf.data.Dataset.save(train_ds, '{}tf_Dataset_train_NeuralOpt_{}x{}_{}_samples.data'.format(outdir,nr,nz, X_train.shape[0]))

In [ ]:
# compute kernels for the GS equation test
y_test = y_test_all_domain
X_test = X_test_all_domain
res_RHS_pixel_test = res_RHS_pixel_test_all_domain
RR_pixel_test = np.tile(RR_pixels,(X_test.shape[0],1,1))
ZZ_pixel_test = np.tile(ZZ_pixels,(X_test.shape[0],1,1))
''' Compute kernels for GS Equation for the qeuils on the sub-grids (dr,dz is grid-dependent), 
otherwise we would have to compute them on the fly during each training step '''
Laplace_kernel = np.zeros((y_test.shape[0],3,3),dtype=DTYPE)
Df_dr_kernel = np.zeros((y_test.shape[0],3,3),dtype=DTYPE)
for i in tqdm(range(len(y_test))):
    hr = RR_pixel_test[i,1,2] - RR_pixel_test[i,1,1]
    hz = ZZ_pixel_test[i,2,1] - ZZ_pixel_test[i,1,1] 
    alfa = -2*(hr**2 + hz**2)
    Laplace_kernel_i = np.array(([0, hr**2/alfa, 0], [hz**2/alfa, 1, hz**2/alfa], [0, hr**2/alfa, 0]))
    Df_dr_kernel_i = np.array(([0, 0, 0], [+1, 0, -1], [0, 0, 0]))/(2*hr*alfa)*(hr**2*hz**2)
    Laplace_kernel[i,:,:] = Laplace_kernel_i
    Df_dr_kernel[i,:,:] = Df_dr_kernel_i
# Dataset with equilibria on the entire domain
train_ds = tf.data.Dataset.from_tensor_slices(( X_test, y_test, res_RHS_pixel_test[:,1:-1,1:-1], RR_pixel_test, ZZ_pixel_test, Laplace_kernel, Df_dr_kernel)).shuffle(42)
tf.data.Dataset.save(train_ds, '{}tf_Dataset_test_NeuralOpt_{}x{}_{}_samples.data'.format(outdir,nr,nz, y_test.shape[0]))